In [1]:
!pip install vaex

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import vaex

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/Fraud.csv')

In [5]:
vaex_df=vaex.from_pandas(df)

In [6]:
type(vaex_df)

vaex.dataframe.DataFrameLocal

In [7]:
vaex_df.export_hdf5("Fraud.hdf5")

In [8]:
vaex_df=vaex.open("Fraud.hdf5")

In [9]:
vaex_df.head()

#,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136,160296,M1979787155,0,0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249,19384.7,M2044282225,0,0,0,0
2,1,TRANSFER,181,C1305486145,181,0,C553264065,0,0,1,0
3,1,CASH_OUT,181,C840083671,181,0,C38997010,21182,0,1,0
4,1,PAYMENT,11668.1,C2048537720,41554,29885.9,M1230701703,0,0,0,0
5,1,PAYMENT,7817.71,C90045638,53860,46042.3,M573487274,0,0,0,0
6,1,PAYMENT,7107.77,C154988899,183195,176087,M408069119,0,0,0,0
7,1,PAYMENT,7861.64,C1912850431,176087,168226,M633326333,0,0,0,0
8,1,PAYMENT,4024.36,C1265012928,2671,0,M1176932104,0,0,0,0
9,1,DEBIT,5337.77,C712410124,41720,36382.2,C195600860,41898,40348.8,0,0


In [10]:
# Check the distribution of 'type' (transaction types)
vaex_df['type'].value_counts()

,0
PAYMENT,96310
CASH_OUT,94925
CASH_IN,57745
TRANSFER,22998
DEBIT,2079


In [11]:
# Check how often fraud occurs
vaex_df['isFraud'].value_counts()

,0
0.0,273887
1.0,169
NaN,1


In [12]:
# Count missing values for important columns
missing_isFraud = vaex_df['isFraud'].isna().sum()
missing_oldbalanceDest = vaex_df['oldbalanceDest'].isna().sum()
missing_newbalanceDest = vaex_df['newbalanceDest'].isna().sum()

print(f"Missing in isFraud: {missing_isFraud}")
print(f"Missing in oldbalanceDest: {missing_oldbalanceDest}")
print(f"Missing in newbalanceDest: {missing_newbalanceDest}")

Missing in isFraud: 1
Missing in oldbalanceDest: 1
Missing in newbalanceDest: 1


In [13]:
# Filter out rows where the specified columns have missing values
vaex_df = vaex_df[~(vaex_df['isFraud'].isna()) & ~(vaex_df['oldbalanceDest'].isna()) & ~(vaex_df['newbalanceDest'].isna())]

In [14]:
# Get unique values in the 'type' column
unique_types = vaex_df['type'].unique()

# Create one-hot encoded columns for each unique value in 'type'
for category in unique_types:
    vaex_df[category] = (vaex_df['type'] == category)

# Drop the original 'type' column after encoding
vaex_df = vaex_df.drop(columns=['type'])

# Check the first few rows after encoding
vaex_df.head()

#,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,TRANSFER,PAYMENT,DEBIT,CASH_OUT
0,1,9839.64,C1231006815,170136,160296,M1979787155,0,0,0,0,False,False,True,False,False
1,1,1864.28,C1666544295,21249,19384.7,M2044282225,0,0,0,0,False,False,True,False,False
2,1,181,C1305486145,181,0,C553264065,0,0,1,0,False,True,False,False,False
3,1,181,C840083671,181,0,C38997010,21182,0,1,0,False,False,False,False,True
4,1,11668.1,C2048537720,41554,29885.9,M1230701703,0,0,0,0,False,False,True,False,False
5,1,7817.71,C90045638,53860,46042.3,M573487274,0,0,0,0,False,False,True,False,False
6,1,7107.77,C154988899,183195,176087,M408069119,0,0,0,0,False,False,True,False,False
7,1,7861.64,C1912850431,176087,168226,M633326333,0,0,0,0,False,False,True,False,False
8,1,4024.36,C1265012928,2671,0,M1176932104,0,0,0,0,False,False,True,False,False
9,1,5337.77,C712410124,41720,36382.2,C195600860,41898,40348.8,0,0,False,False,False,True,False


In [15]:
# Create new features
vaex_df['balanceDeltaOrig'] = vaex_df['oldbalanceOrg'] - vaex_df['newbalanceOrig']
vaex_df['balanceDeltaDest'] = vaex_df['oldbalanceDest'] - vaex_df['newbalanceDest']

In [16]:
# List of numeric columns to scale
numeric_columns = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# Apply min-max scaling in Vaex
for col in numeric_columns:
    min_val = vaex_df[col].min()
    max_val = vaex_df[col].max()
    vaex_df[col + '_scaled'] = (vaex_df[col] - min_val) / (max_val - min_val)

In [17]:
# Separate features and target variable
features = vaex_df.drop(columns=['isFraud'])
target = vaex_df['isFraud']

In [18]:
# Clear any filters in the DataFrame
vaex_df = vaex_df.extract()  # This ensures no filtering is applied

# Now you can perform train-test split
train_vaex, test_vaex = vaex_df.ml.train_test_split(test_size=0.2)

# Check the sizes of the training and test sets
print(f"Training set length: {len(train_vaex)}")
print(f"Test set length: {len(test_vaex)}")

Training set length: 219245
Test set length: 54811


/usr/local/lib/python3.10/dist-packages/vaex/ml/__init__.py:28: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [19]:
import vaex
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from vaex.ml.sklearn import Predictor

In [20]:
# Select features (excluding 'isFraud')
features = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',
            'CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']  # Use the correct one-hot columns

# Target
target = 'isFraud'

# Train and test sets
X_train = train_vaex[features].to_pandas_df()  # Convert Vaex to Pandas for compatibility
y_train = train_vaex[target].to_numpy()  # Get target variable as a NumPy array

X_test = test_vaex[features].to_pandas_df()  # Convert Vaex to Pandas for compatibility
y_test = test_vaex[target].to_numpy()  # Get target variable as a NumPy array

In [21]:
# Initialize the Logistic Regression model
log_reg = LogisticRegression()

# Train the model
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Logistic Regression - \n Accuracy: {accuracy}, \n Precision: {precision}, \n Recall: {recall}, \n F1 Score: {f1}")

Logistic Regression - 
 Accuracy: 0.9984309718852056, 
 Precision: 0.9375, 
 Recall: 0.15, 
 F1 Score: 0.25862068965517243


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest - \n Accuracy: {accuracy_rf}, \n Precision: {precision_rf}, \n Recall: {recall_rf}, \n F1 Score: {f1_rf}")

Random Forest - 
 Accuracy: 0.99859517250187, 
 Precision: 1.0, 
 Recall: 0.23, 
 F1 Score: 0.37398373983739835


In [23]:
# Use Vaex's sklearn Predictor to integrate the model with Vaex
log_reg_predictor = Predictor(model=log_reg, features=features, target=target)

# Attach the model to the training dataset for future predictions
train_vaex = log_reg_predictor.transform(train_vaex)
test_vaex = log_reg_predictor.transform(test_vaex)

# Predict on Vaex DataFrame
predictions = train_vaex['prediction'].values  # Get predictions from the attached model

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [24]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Convert Vaex DataFrames to Pandas DataFrames
X_train = train_vaex[features].to_pandas_df()
y_train = train_vaex[target].to_numpy()

X_test = test_vaex[features].to_pandas_df()
y_test = test_vaex[target].to_numpy()

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True)
}

# Dictionary to hold results
results = {}

# Train, predict and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Store results
    results[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:21:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


                     Accuracy  Precision  Recall  F1 Score
Logistic Regression  0.998431   0.937500    0.15  0.258621
Random Forest        0.998632   1.000000    0.25  0.400000
XGBoost              0.998741   0.942857    0.33  0.488889
SVM                  0.998212   1.000000    0.02  0.039216
